# Dataset

Asagida verilen dataseti tasniflendirmek icin bir modele ihtiyac duyulmaktadir. Sekilde de goruldugu uzere, siniflari ayiran bir dogru, feature uzayinda mevcut degildir.

Problemin cozumu icin bir sinir agi kullanacagiz.

In [ ]:
from ml_utils import make_sekil
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X, y = make_sekil(span=8, samples=200, n_redundant=0, random_state=42)
y = np.array(y)[:, None]

In [ ]:
plt.scatter(*X.T, c=y)

# Sinir Agi Parcalari

Sinir agi mimarisini, zincir turev kuralini ve donguleri bildigimize gore artik kendi sinir agimizi sifirdan kodlayacak tum donanima sahibiz.

Sinir agimiz sirasiyla asagidaki alt rutinlere ihtiyac duyacaktir:
- `activation_function`: Sinir agimizin ihtiyac duydugu hayati non-lineer donusum
- `initialize_weights`: Verilen mimariye gore ilk agirliklari olusturma
- `forward`: Verilen datayi ve katsayilari kullanarak bir *feed-forward* islemi
- `backpropagate`: Zincir seklinde turevleri hesaplayip katsayilari guncelleme islemi

## Isinma: Aktivasyon

Sinir agimizi lineer regresyondan ayiran en onemli seyin non-lineer aktivasyonlar oldugunu gormustuk. Simdi kullanilabilecek aktivasyon turlerinden biri olan `sigmoid` aktivasyon fonksiyonunu kodlayalim:

In [ ]:
def sigmoid(x):
    return None

## Agirliklari ve interseptleri baslatma

In [ ]:
def initialize_weights(input_size, hidden_layers, output_size):
    """Verilen mimariye gore katmanların ağırlıklarının initialize edilmesi.

    Girdiler:
    ----
    input_size: int
        Girdi katmanın boyutu. Datasetimizdeki feature sayisi.
    hidden_layers: List[int]
        Gizli katmanların boyutları. Ornegin, [3, 4, 2] gibi...
    output_size: int
        Çıktı katmanın boyutu. Datasetimizdeki target sayisi.

    Cikti:
    ----
    betas: List[np.array]
        Gizli katmanların ağırlıkları.
    intercepts: List[np.array]
        Gizli katmanların interceptleri.
    
    """
    input_layer = input_size
    output_layer = output_size

    betas = [None] * (len(hidden_layers) + 1) # +1 cikti katmani icin
    intercepts = [None] * (len(hidden_layers) + 1) # her katsayi matrisi icin bir de intersepte ihtiyac var
    n = len(betas) + 1


    # Baslangic kosullari:
    betas[0] = np.random.randn(None, hidden_layers[0])  # ilk katsayi matrisinin boyutu kac olmali?

    # her intersept degeri, bagli oldugu katsayi matrisinin ikinci boyutu
    # buyuklugunde olmalidir.
    intercepts[0] = np.random.randn(hidden_layers[0])

    # Aradaki katsayilar:
    for i in range(1, n-2):
        betas[i] = np.random.randn(None, None) # dogru boyutlari belirtiniz
        intercepts[i] = np.random.randn(None) # dogru boyutu belirtiniz
    
    # Bitis:
    betas[n - 2] = np.random.randn(None, None)  # Son gizli aktivasyonu ciktiya goturecek agirlik boyutu nedir?
    intercepts[n - 2] = np.random.randn(None)  # Cikti icin dogru intersept boyutunu giriniz.

    # Dogru donusleri, dokumentasyondaki sirada yapiniz.
    return None, None

## Feed-forward

Feed-forward icin asagidaki notasyonu kullanabiliriz:

$$a_0 = X$$

$$ z_1 = X \beta_0 + sabit_0$$
$$ a_1 = \sigma(z_1)$$
$$ ... $$
$$ z_i = a_{i-1} \beta_{i-1} + sabit_{i - 1} $$
$$ a_i = \sigma(z_i)$$
$$ ... $$
$$ \hat{y} = a_{n-1}$$


In [ ]:
def forward(X, y, betas, intercepts, reg=0.0):
    """Verilen girdi ve ağırlıklari kullanarak ileri besleme yapar.
    
    Girdiler:
    ----
    X: np.array
        Girdi verileri.
    y: np.array
        Cikti verileri.
    betas: List[np.array]
        Gizli katmanların ağırlıkları.
    intercepts: List[np.array]
        Gizli katmanların interceptleri.
    reg: float
        Regularizasyon katsayisi. Maliyet hesabinda kullanilir.

    Cikti:
    ----
    yhat: np.array
        Ileri besleme sonucu.
    a: List[np.array]
        Gizli katmanların aktivasyonları.
    J: float
        Maliyet.
    """
    n = len(betas) + 1
    a = [None] * n
    z = [None] * n

    a[0] = None # ilk aktivasyon icin dogru baslangic degerini ayarlayiniz
    for i in range(1, n):
        z[i] = None # yukaridaki formule gore uygun ifadeyi yaziniz.
        a[i] = None # Uygun aktivasyonu yapiniz.

    yhat = a[n-1]
    J = np.sum(-y * np.log(yhat) - (1 - y) * np.log(1 - yhat)) + reg * sum((betas[i] ** 2).sum() for i in range(n-1))

    # Dokumentasyona uygun sirada dogru degiskenleri dondurunuz.
    # Geri yayilimda ihtiyacimiz olacagi icin hesaplanan aktivasyonlari dondurmeyi unutmayiniz.
    return None, None, None


## Backpropagation

Geri yayilim icin turevleri zincir seklinde geri dogru hesaplayabiliriz.

Bu maksatla asagidaki notasyonu kullanabiliriz:

Baslangic kosulu:
$$ a_{n-1} = \hat{y} $$
olduguna gore
$$\frac{dJ}{da_{n-1}} = \frac{1-y}{1 - a_{n-1}} - \frac{y}{a_{n-1}}$$
$$ \frac{dJ}{dz_{n-1}} = a_{n-1} (1 - a_{n-1}) \frac{dJ}{da_{n-1}} $$

`n-2`'den `0`'a kadar:

$$ \frac{dJ}{da_i} = \frac{dJ}{dz_{i+1}} \beta_i^T $$
$$ \frac{dJ}{dz_i} = a_i (1 - a_i) \frac{dJ}{da_i}$$

### Guncelleme kurallari

#### Turevler

`i=0`'dan `len(beta)`'ya kadar:

$$ \frac{dJ}{d\beta_i} = a_i^T \frac{dJ}{dz_{i + 1}} + \lambda \beta_i$$
$$ \frac{dJ}{dsabit_i} = 1 ^T \frac{dJ}{dz_{i + 1}}$$

#### Soru: Guncellemeler

$\beta_i$ ve $sabit_i$ icin, elinizdeki turevleri ve ogrenme orani $\alpha$'yi kullanarak guncelleme formulunu yaziniz:


$$ formulunuzu\ buraya\ yaziniz$$

In [ ]:
def backpropagate(betas, intercepts, a, y, lr, reg=0.0):
    n = len(betas) + 1

    da = [None] * n
    dz = [None] * n
    dint = [None] * n

    ## Baslangic kosullari:
    # Yukaridaki notasyonlardan uygun baslangic kosulunu bulup doldurunuz.
    da[n - 1] = None
    dz[n - 1] = None


    # Aktivasyon ve lineer kisimlarin turevleri
    ## Dongude her da[i] ve dz[i] degeri icin dogru ifadeleri notasyondan yararlanarak yaziniz.
    for i in range(n-2, -1, -1):
        da[i] = None
        dz[i] = None

    dbeta = [None] * len(betas)

    for i in range(len(betas)):
        # Her beta[i] parametresi icin turevi hesaplayacak uygun ifadeyi yaziniz. 
        dbeta[i] = None  # Opsiyonel regularizasyon kismina dikkat ediniz.
        dint[i] = dz[i + 1].sum(axis=0)  # 1'ler matrisiyle carpmayi bu sekilde yorumlayabilirsiniz.

    for i in range(len(betas)):
        # Hesaplmais oldugunuz turevleri kullanarak parametreleri uygun sekilde guncelleyiniz.
        # i'nci parametre betas[i]'nin turevine dbetas[i] ile ulasabilirsiniz.
        betas[i] = None

        # i'nci intersept intercepts[i]'nin turevine dint[i] ile ulasabilirsiniz.
        intercepts[i] = None

    return betas, intercepts

    

#### Algoritmamizi test edelim

In [ ]:
betas, intercepts = initialize_weights(input_size=2, hidden_layers=[20, 20, 20], output_size=1)
LAMDA = 0.01

In [ ]:
for i in range(1000):
    yhat, a, J = forward(X, y, betas, intercepts, reg=LAMDA)
    betas, intercepts = backpropagate(betas, intercepts, a, y, lr=0.001, reg=LAMDA)

In [ ]:
xx, yy = np.meshgrid(np.linspace(-8, 8, 100), np.linspace(-8, 8, 100))
X_grid = np.c_[xx.ravel(), yy.ravel()]
y_grid = forward(X_grid, X_grid, betas, intercepts)[0].reshape(xx.shape)
plt.contourf(xx, yy, y_grid, cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(*X.T, c=y)

## Sinir Agi Class'i

Bir `class` yazarak, sinir agi agirliklarini ve interseptlerini ortalikta dolastirmadan, tek bir yapi icerisinde tutabiliriz.

In [ ]:
class SinirAgi:
    def __init__(self, input_size, hidden_layers, output_size, lr=0.01, reg=0.0):

        # ALinan argumanlardan da yararlanarak, attribute'leri uygun sekilde atayiniz.
        self.betas, self.intercepts = None  # Agirlik ve interseptleri rastgele baslatacak fonksiyonu kullaniniz
        self.lr = None
        self.reg = None

        # Diyagnostik icin
        self.loss = []
        self.accuracy = []

    def fit(self, X, y, epochs=100):
        if y.ndim == 1:
            y = y[:, None]

        for i in range(epochs):
            # Uygun alt rutinleri kullaniniz:
            yhat, a, J = None  # Feedforward islemini gerceklestirecek alt rutini kullaniniz.
            self.betas, self.intercepts = # Backpropagation islemini gerceklestirecek alt rutini kullaniniz.
            self.loss.append(J)
            self.accuracy.append(np.mean(y == yhat))

    def predict(self, X):
        # Alt rutinin argumanlarini uygun sekilde doldurunuz.
        yhat, a, _ = forward(X, X, betas=None, intercepts=None, reg=None, lr=None)
        return yhat

In [ ]:
# Sinir agi nesnenizi tanimlayin:
# Girdi katman buyuklugu 2
# Tek gizli katman: 30
# Cikti katman buyuklugu 1
# Ogrenme orani 0.001
# Regularizasyon 0.01

sa = None

In [ ]:
# Sinir aginizi X ve y degiskenleri uzerinde 1000 iterasyon boyunca egitiniz.
None

In [ ]:
xx, yy = np.meshgrid(np.linspace(-8, 8, 100), np.linspace(-8, 8, 100))
X_grid = np.c_[xx.ravel(), yy.ravel()]
y_grid = sa.predict(X_grid).reshape(xx.shape)

plt.figure(figsize=(16, 4))
plt.subplot(1, 2, 1)
plt.contourf(xx, yy, y_grid, cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(*X.T, c=y)

plt.subplot(1, 2, 2)
plt.plot(sa.loss)

### Sinir Agi Class'ni seffaflastirma

Hazirlamis oldugunuz `SinirAgi` class'inin gizli katman ciktilarini inceleyebilmek istiyorsunuz.

Bu maksatla `SeffafSinirAgi` isimli bir alt sinif olusturup `aktivasyon_al` metodunu ekleyebiliriz.

`.aktivasyon_al(X, i)` metodu, verilen data icin agimizdaki `i`'nci aktivasyonu vize geri dondurmeli.
> `a[0] = X` olduguna dikkat ediniz.

In [ ]:
class SeffafSinirAgi(SinirAgi):
    def aktivasyon_al(self, X, i):
        """i'nci gizli katmanin aktivasyonunu getir.
        
        Argumanlar:
            X: np.ndarray, X değerleri
            i: int, aktivasyonunu getirilecek gizli katmanın indisi
        Donus:
            aktivasyondegeri: np.ndarray, aktivasyon değerleri

        Notlar:
            a[0] = X olduguna dikkat ediniz. i = 0 verilirse fonksiyon X'i döndürür.
        """
        yhat, a, _ = forward(X, X, self.betas, self.intercepts, self.reg)
        # Uygun degeri seciniz.
        aktivasyondegeri = None
        return aktivasyondegeri

Kurmus oldugumuz alt sinifi asagida test edelim:

In [ ]:
ssa = SeffafSinirAgi(2, [30, 30, 3], 1, lr=0.01, reg=0.001)
ssa.fit(X, y, 1000)

xx, yy = np.meshgrid(np.linspace(-8, 8, 100), np.linspace(-8, 8, 100))
X_grid = np.c_[xx.ravel(), yy.ravel()]
y_grid = ssa.predict(X_grid).reshape(xx.shape)

plt.figure(figsize=(16, 4))
plt.subplot(1, 2, 1)
plt.contourf(xx, yy, y_grid, cmap=plt.cm.coolwarm, alpha=0.8)
plt.scatter(*X.T, c=y)

plt.subplot(1, 2, 2)
plt.plot(ssa.loss);

Gizli katman `3`'un gorsellestirilmesi icin 3 boyutlu bir *scatterplot* kullaniniz.

> Neden 3 boyutlu bir gorsele ihtiyacimiz var?

In [ ]:
%matplotlib qt5
None